In [1]:
%load_ext autoreload
%autoreload 2
from shot.models import Period, Year, Month, Week, Day, Session, Shot, SonyData, SessionLabel
from video.models import VideoSource
from django.contrib.auth.models import User
from profiles.models import UserProfile
from shot.import_sony import csv2shotsvideos
#from passlib.hash import pbkdf2_sha256
from shot.tasks import sony_csv_to_db
import datetime as dt

In [4]:
user = User(username='ales')
user.save()

In [13]:
year = Year.objects.all()[0]
year.delete()

(6453,
 {'shot.Day': 8,
  'shot.Month': 2,
  'shot.Period': 19,
  'shot.Session': 8,
  'shot.Session_labels': 0,
  'shot.Shot': 3190,
  'shot.SonyData': 3190,
  'shot.Year': 1,
  'video.VideoSource': 35})

In [2]:
user = User.objects.get(username='ales')
user, user.pk, Year.objects.count(), Month.objects.count(), Week.objects.count(), Day.objects.count(), Session.objects.count(), Shot.objects.count(), SonyData.objects.count(), VideoSource.objects.count()

(<User: ales>, 2, 1, 2, 4, 8, 8, 3190, 3190, 35)

In [8]:
shot = user.shots.all()[0]
shot.data.swing_type

'FH'

In [6]:
pks = sony_csv_to_db('/home/alessandro/Documents/Sony_Sensor/20161208123845.csv',2)

/home/alessandro/webapps/tennistat2/venv/lib/python3.5/site-packages/django/db/models/fields/__init__.py:1372: RuntimeWarning: DateTimeField Period.timestamp received a naive datetime (2016-01-01 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/alessandro/webapps/tennistat2/venv/lib/python3.5/site-packages/django/db/models/fields/__init__.py:1372: RuntimeWarning: DateTimeField Period.timestamp received a naive datetime (2016-11-01 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/alessandro/webapps/tennistat2/venv/lib/python3.5/site-packages/django/db/models/fields/__init__.py:1372: RuntimeWarning: DateTimeField Period.timestamp received a naive datetime (2016-11-21 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/alessandro/webapps/tennistat2/venv/lib/python3.5/site-packages/django/db/models/fields/__init__.py:1372: RuntimeWarning: DateTimeField Period.timestamp received a naive datetime (2016-11-22 00:00:00) while time zone s

In [12]:
shots_videos = csv2shotsvideos('/home/alessandro/Videos/SonySensor/20161228194109.csv')
len(shots_videos)

3225

In [3]:
day = Day.objects.all()[1]
day.videosource_set.count()

7

In [13]:
len(shots_videos[shots_videos['video_filename'].notnull()]), len(shots_videos)

(35, 3225)

In [5]:
day = Day.objects.filter(user=user)[3]
session = day.session_set.all()[0]

In [4]:
label = SessionLabel()
label.name = 'Singles'
label.user = user
label.save()

In [12]:
session.labels.add(label)

In [14]:
label.session_set.add(session)

In [16]:
Session.objects.filter(user=user)

<QuerySet [<Session: 2016/11/22/18>, <Session: 2016/11/23/17>, <Session: 2016/11/28/18>, <Session: 2016/12/1/19>, <Session: 2016/12/2/17>, <Session: 2016/12/5/20>, <Session: 2016/12/7/18>, <Session: 2016/12/28/17>]>

In [102]:
from api.serializers import DaySerializer, LabelSerializer, SessionSerializer

data = {"user":2,"name":"Dingles","slug":"asd"}
ll = LabelSerializer(lab,data=data)
ll.is_valid()

True

In [104]:
ll.validated_data

OrderedDict([('user', <User: ales>), ('name', 'Dingles'), ('slug', 'asd')])

In [93]:
instance.pk

6

In [105]:
lab = SessionLabel.objects.all()[3]
lab.pk, lab.user, lab.name, lab.slug

(4, <User: ales>, 'Dingles', 'dingles')

In [135]:
ss = SessionSerializer(session)
data = ss.data
data

ReturnDict([('pk', 34),
            ('user', 2),
            ('timestamp', '2016-12-01T19:44:37Z'),
            ('shot_count', 570),
            ('video_count', 4),
            ('labels',
             [OrderedDict([('pk', 1),
                           ('user', 2),
                           ('name', 'Singles'),
                           ('slug', 'singles')])])])

In [136]:
data['labels'].append(LabelSerializer(label).data)

In [7]:
shot = day.shots.all()[100]


In [22]:
days = Day.objects.filter()[0:3]

/home/alessandro/webapps/tennistat2/venv/lib/python3.5/site-packages/django/db/models/base.py:310: RuntimeWarning: Model 'shot.period' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)
/home/alessandro/webapps/tennistat2/venv/lib/python3.5/site-packages/django/db/models/base.py:310: RuntimeWarning: Model 'shot.year' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)
/home/alessandro/webapps/tennistat2/venv/lib/python3.5/site-packages/django/db/models/base.py:310: RuntimeWarning: Model 'shot.month' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_cl

In [31]:
for day in days:
    print(day, day.pk)

2016/11/22 26
2016/11/23 28
2016/11/28 30


In [95]:
the_days = Day.objects.filter(user=user,pk__in=[26,28,30])
from django.db.models import Q
#QS = []
QS = Q(user__username='admin')
QS.add(Q(day__in=the_days), Q.AND)
#QS.add(Q(user=user), Q.AND)
#que.append(Q(user=user))
QS

SyntaxError: keyword can't be an expression (<ipython-input-95-ac90c04c9c74>, line 5)

In [93]:
Shot.objects.filter(QS).count()

0

In [45]:
from api.objects import SonyFilter
input_dict = {
"username":"ales",
"sensor":"SO",
"filters":{
	"periods":{
		"name":"day",
		"pks":[30,33,35]
		},
	"date_range":{
		"start":"yyyy-mm-dd",
		"end":"yyyy-mm-dd"
		},
	"swing_speed":{
		"min":0,
		"max":1000
		},
	"ball_speed":{
		"min":0,
		"max":1000
		},
	"ball_spin":{
		"min":-10,
		"max":10
		},
	"swing_type":[]
	}
}

In [51]:
obj = SonyFilter(input_dict)

In [53]:
obj.filter

{'filters': {'ball_speed': {'max': 1000, 'min': 0},
  'ball_spin': {'max': 10, 'min': -10},
  'periods': {'name': 'day', 'pks': [30, 33, 35]},
  'swing_speed': {'max': 1000, 'min': 0}},
 'sensor': 'SO',
 'username': 'ales'}

In [110]:
end = dt.datetime.strptime('2016-12-1'+'T23:59:59','%Y-%m-%dT%H:%M:%S')

end


datetime.datetime(2016, 12, 1, 23, 59, 59)